In [1]:
import pandas as pd
from pandas.io.json import json_normalize
import json
pd.options.display.max_rows = 6000

neighborhoods_geo_nest = pd.read_json('neighbourhoods.geojson')

In [2]:
# columns

neighborhoods_geo_nest.columns

Index(['type', 'features'], dtype='object')

In [3]:
# first 5 rows of calendar

neighborhoods_geo_nest.head()

,type,features
0,FeatureCollection,"{'type': 'Feature', 'geometry': {'type': 'Mult..."
1,FeatureCollection,"{'type': 'Feature', 'geometry': {'type': 'Mult..."
2,FeatureCollection,"{'type': 'Feature', 'geometry': {'type': 'Mult..."
3,FeatureCollection,"{'type': 'Feature', 'geometry': {'type': 'Mult..."
4,FeatureCollection,"{'type': 'Feature', 'geometry': {'type': 'Mult..."


In [4]:
# flatten nested neighborhoods geo-data

feature = []
neighborhood_group = []
neighborhood = []
point = []
coordinates = []

feature_list = neighborhoods_geo_nest['features']
total_features = len(feature_list)

for i in range(total_features):
    coords_per_feature_i = len(feature_list[i]['geometry']['coordinates'][0][0])
    for j in range(coords_per_feature_i):
        featurei_j = i
        neighborhoodgroupi_j = feature_list[i]['properties']['neighbourhood_group']
        neighborhoodi_j = feature_list[i]['properties']['neighbourhood']
        coordsi_j = feature_list[i]['geometry']['coordinates'][0][0][j]
        feature.append(featurei_j)
        neighborhood_group.append(neighborhoodgroupi_j)        
        neighborhood.append(neighborhoodi_j)
        point.append(j)
        coordinates.append(coordsi_j)

neighbohoods_geo_flat = pd.DataFrame([feature, neighborhood_group, neighborhood, point, coordinates]).T
neighbohoods_geo_flat.columns = ['feature', 'neighborhood_group', 'neighborhood', 'point', 'coordinates']
neighbohoods_geo_flat

,feature,neighborhood_group,neighborhood,point,coordinates
0,0,Ciutat Vella,el Raval,0,"[2.177386, 41.375347]"
1,0,Ciutat Vella,el Raval,1,"[2.178532, 41.374845]"
2,0,Ciutat Vella,el Raval,2,"[2.178881, 41.374692]"
3,0,Ciutat Vella,el Raval,3,"[2.178217, 41.373863]"
4,0,Ciutat Vella,el Raval,4,"[2.183148, 41.371629]"
...,...,...,...,...,...
16698,74,SarriÃ -Sant Gervasi,"Vallvidrera, el Tibidabo i les Planes",123,"[2.067025, 41.426859]"
16699,74,SarriÃ -Sant Gervasi,"Vallvidrera, el Tibidabo i les Planes",124,"[2.067195, 41.426316]"
16700,74,SarriÃ -Sant Gervasi,"Vallvidrera, el Tibidabo i les Planes",125,"[2.069735, 41.425756]"
16701,74,SarriÃ -Sant Gervasi,"Vallvidrera, el Tibidabo i les Planes",126,"[2.071304, 41.42298]"


In [5]:
# determining missingness among columns

def missingness(df):
    return df.isna().sum().sort_values(ascending = False) / len(df) * 100

missingness(neighbohoods_geo_flat)

coordinates           0.0
point                 0.0
neighborhood          0.0
neighborhood_group    0.0
feature               0.0
dtype: float64

In [6]:
# starting column dtypes

neighbohoods_geo_flat.dtypes

feature               object
neighborhood_group    object
neighborhood          object
point                 object
coordinates           object
dtype: object

In [7]:
neighbohoods_geo_flat = neighbohoods_geo_flat.astype({'feature': 'int64', 'point': 'int64'})
neighbohoods_geo_flat.dtypes

feature                int64
neighborhood_group    object
neighborhood          object
point                  int64
coordinates           object
dtype: object

In [8]:
neighbohoods_geo_flat.head()

,feature,neighborhood_group,neighborhood,point,coordinates
0,0,Ciutat Vella,el Raval,0,"[2.177386, 41.375347]"
1,0,Ciutat Vella,el Raval,1,"[2.178532, 41.374845]"
2,0,Ciutat Vella,el Raval,2,"[2.178881, 41.374692]"
3,0,Ciutat Vella,el Raval,3,"[2.178217, 41.373863]"
4,0,Ciutat Vella,el Raval,4,"[2.183148, 41.371629]"


In [9]:
neighbohoods_geo_flat.to_csv('neighbohoods_geo-clean.csv', index = False)

In [10]:
neighbohoods_geo_flat.to_json('neighbohoods_geo.json', orient = 'split', index = False)